### 기본 Retriever에서 여러 테스트를 하기 위함

In [3]:
import os
import nest_asyncio
from dotenv import load_dotenv

load_dotenv()
nest_asyncio.apply()

In [5]:
data_path = "/home/jun/my_project/langchain_tutorial/Langcahin_Tutoraial/data/약관-48-51.pdf"

# Chroma

In [4]:
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_chroma import Chroma


def create_retriever(file_path):
    # 단계 1: 문서 로드(Load Documents)
    # 파서 설정
    parser = LlamaParse(
        result_type="markdown",  # "markdown"과 "text" 사용 가능
        num_workers=8,  # worker 수 (기본값: 4)
        verbose=True,
        language="ko",
    )
    # SimpleDirectoryReader를 사용하여 파일 파싱
    file_extractor = {".pdf": parser}

    # LlamaParse로 파일 파싱
    documents = SimpleDirectoryReader(
        input_files=[file_path],
        file_extractor=file_extractor,
    ).load_data()

    # 랭체인 도큐먼트로 변환
    docs = [doc.to_langchain_format() for doc in documents]

    # 단계 2: 문서 분할(Split Documents)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    split_documents = text_splitter.split_documents(docs)

    # 단계 3: 임베딩(Embedding) 생성
    model_name = "BAAI/bge-m3"
    model_kwargs = {"device":"cpu"}
    encode_kwargs = {"normalize_embeddings":True}

    embeddings = HuggingFaceBgeEmbeddings(
        model_name=model_name, model_kwargs=model_kwargs,encode_kwargs=encode_kwargs)

    # 단계 4: DB 생성(Create DB) 및 저장
    # 벡터스토어를 생성합니다.
        # 이게 데모용(FAISS)으로 빠르게 확인하기 좋아서 쓰는거지, 다른걸로 꼭 봐꿔줘야 함
    vectorstore = Chroma.from_documents(documents=split_documents, embedding=embeddings)

    # 단계 5: 검색기(Retriever) 생성
    # 문서에 포함되어 있는 정보를 검색하고 생성합니다.
    retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
    return retriever

In [7]:
%%time
chroma_retriever = create_retriever(data_path)

Started parsing the file under job_id 8eabe9a4-15dd-4478-b100-a7ea7e2d3266
CPU times: user 1min 21s, sys: 3.7 s, total: 1min 25s
Wall time: 21.2 s


In [12]:
qa = chroma_retriever.invoke("간병인지원 상해입원비의")
print(qa)

# FAISS

In [9]:
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.vectorstores import FAISS


def create_retriever(file_path):
    # 단계 1: 문서 로드(Load Documents)
    # 파서 설정
    parser = LlamaParse(
        result_type="markdown",  # "markdown"과 "text" 사용 가능
        num_workers=8,  # worker 수 (기본값: 4)
        verbose=True,
        language="ko",
    )
    # SimpleDirectoryReader를 사용하여 파일 파싱
    file_extractor = {".pdf": parser}

    # LlamaParse로 파일 파싱
    documents = SimpleDirectoryReader(
        input_files=[file_path],
        file_extractor=file_extractor,
    ).load_data()

    # 랭체인 도큐먼트로 변환
    docs = [doc.to_langchain_format() for doc in documents]

    # 단계 2: 문서 분할(Split Documents)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    split_documents = text_splitter.split_documents(docs)

    # 단계 3: 임베딩(Embedding) 생성
    model_name = "BAAI/bge-m3"
    model_kwargs = {"device":"cpu"}
    encode_kwargs = {"normalize_embeddings":True}

    embeddings = HuggingFaceBgeEmbeddings(
        model_name=model_name, model_kwargs=model_kwargs,encode_kwargs=encode_kwargs)

    # 단계 4: DB 생성(Create DB) 및 저장
    # 벡터스토어를 생성합니다.
        # 이게 데모용(FAISS)으로 빠르게 확인하기 좋아서 쓰는거지, 다른걸로 꼭 봐꿔줘야 함
    vectorstore = FAISS.from_documents(documents=split_documents, embedding=embeddings)

    # 단계 5: 검색기(Retriever) 생성
    # 문서에 포함되어 있는 정보를 검색하고 생성합니다.
    retriever = vectorstore.as_retriever(search_kwargs={"k": 2})
    return retriever

In [10]:
%%time
FAISS_retriever = create_retriever(data_path)

Started parsing the file under job_id 77fd3484-b22d-4045-b48d-2bbb73806d45
CPU times: user 2min 7s, sys: 32.1 s, total: 2min 40s
Wall time: 38.1 s


In [11]:
FAISS_retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7fccfe981cd0>, search_kwargs={'k': 2})